# Knowledge Graph Querying: Competency Questions (CQs)
This notebook shows how to query KGs to answer the CQs related to the OntoBOT ontology.

We will:
- load local RDF graphs
- Formulate SPARQL queries corresponding to CQ

#### 🔧 Setup 🔧

In [22]:
from rdflib import Graph, Namespace
from collections import defaultdict
from rdflib.namespace import RDFS, FOAF, DCTERMS

import pandas as pd

In [42]:
# Load graph
g = Graph()
g.parse("./scenarios_kgs.ttl", format="ttl")
g.parse("./case_study/hi-thesaurus.ttl", format="ttl")
g.parse("../hi-ontology/hi-ontology.ttl", format="ttl")

<Graph identifier=Neb80cfe3d8724553b466e9f92f04b3f9 (<class 'rdflib.graph.Graph'>)>

In [43]:
HI = Namespace("https://w3id.org/hi-ontology#")
DUL = Namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
HINT = Namespace("http://w3id.org/hi-thesaurus/")
HI_S1 = Namespace("https://w3id.org/hi-ontology/scenarios/1")
HI_S2 = Namespace("https://w3id.org/hi-ontology/scenarios/2")
HI_S3 = Namespace("https://w3id.org/hi-ontology/scenarios/3")
HI_S4 = Namespace("https://w3id.org/hi-ontology/scenarios/4")
HI_S5 = Namespace("https://w3id.org/hi-ontology/scenarios/5")
HI_S6 = Namespace("https://w3id.org/hi-ontology/scenarios/6")
HI_S7 = Namespace("https://w3id.org/hi-ontology/scenarios/7")

g.bind("hi", HI)
g.bind("dul", DUL)
g.bind("hint", HINT)
g.bind("hi-s", HI_S1)
g.bind("hi-s", HI_S2)
g.bind("hi-s", HI_S3)
g.bind("hi-s", HI_S4)
g.bind("hi-s", HI_S5)
g.bind("hi-s", HI_S6)
g.bind("hi-s", HI_S7)

#### 🤖 CQ 1 🤖

**Given a use case, how is the hybrid intelligence team composed, and what roles do its agents assume?**

In [44]:
cq1 = """
PREFIX hi:    <https://w3id.org/hi-ontology#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>
PREFIX hi-s1: <https://w3id.org/hi-ontology/scenarios/1/>

SELECT DISTINCT ?agentType ?agentTypeLabel ?roleConcept ?roleLabel ?roleDefinition
WHERE {
  VALUES ?useCase { hi-s1:UseCasePersonalAssistantHI }
  ?useCase hi:hasHITeam ?team .
  ?team hi:hasMember ?member .
  ?member a ?agentType ;
      hi:hasRoleConcept ?roleConcept .
  ?agentType rdfs:label ?agentTypeLabel .
  ?roleConcept skos:prefLabel ?roleLabel ;
      skos:definition ?roleDefinition .
}
"""

results_cq1 = g.query(cq1)
df_cq1 = pd.DataFrame([(str(row.agentType), str(row.agentTypeLabel), str(row.roleConcept), str(row.roleLabel), str(row.roleDefinition)) for row in results_cq1],
                        columns=["Agent", "Agent Label", "Role Concept", "Role Label", "Role Definition"])
df_cq1

,Agent,Agent Label,Role Concept,Role Label,Role Definition
0,https://w3id.org/hi-ontology#ArtificialAgent,Artificial Agent,https://w3id.org/hi-thesaurus/AIAssistantRole,System Assistant Role,A role where the agent assists the system in p...
1,https://w3id.org/hi-ontology#HumanAgent,Human Agent,https://w3id.org/hi-thesaurus/HumanUserRole,User Role,A role where the agent represents the user int...


#### 🤖 CQ 2 🤖

**Given an agent's capabilities, which tasks can it perform, and in which use cases can it participate?**

In [45]:
cq2 = """
PREFIX hi:    <https://w3id.org/hi-ontology#>
PREFIX hint:  <https://w3id.org/hi-thesaurus/>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?task ?taskLabel ?taskDefinition ?useCase ?useCaseConcept ?useCaseLabel ?useCaseDefinition

WHERE {
  VALUES ?availableCapConcept { hint:KnowledgeGraphQueryingCapability hint:ExplainingCapability hint:SituationalAwarenessCapability }

  ?capability hi:hasCapabilityConcept ?availableCapConcept ;
      hi:allowsTask ?task .
  ?task hi:hasTaskConcept ?taskConcept ;
      hi:requiresCapability ?reqCap .
  ?taskConcept skos:prefLabel  ?taskLabel ;
      skos:definition ?taskDefinition .
  ?reqCap hi:hasCapabilityConcept ?reqCapConcept .

  FILTER NOT EXISTS {
    ?task hi:requiresCapability ?missingCap .
    ?missingCap hi:hasCapabilityConcept ?missingCapConcept .
    FILTER ( ?missingCapConcept NOT IN ( hint:KnowledgeGraphQueryingCapability, hint:ExplainingCapability, hint:SituationalAwarenessCapability ))
  }

  ?agent hi:hasCapability ?capability .
  ?useCase hi:hasHITeam ?team ;
      hi:hasUseCaseConcept ?useCaseConcept .
  ?team hi:hasMember ?agent .
  ?useCaseConcept skos:prefLabel  ?useCaseLabel ;
      skos:definition ?useCaseDefinition .
}
"""

results_cq2 = g.query(cq2)
df_cq2 = pd.DataFrame([(str(row.task), str(row.taskLabel), str(row.taskDefinition), str(row.useCase), 
                        str(row.useCaseConcept), str(row.useCaseLabel), str(row.useCaseDefinition)) for row in results_cq2],
                        columns=["Task", "Task Label", "Task Definition", "Use Case", "Use Case Concept", "Use Case Label", "Use Case Definition"])
df_cq2

,Task,Task Label,Task Definition,Use Case,Use Case Concept,Use Case Label,Use Case Definition
0,https://w3id.org/hi-ontology/scenarios/2/TaskK...,Knowledge Query Task,A task that involves querying knowledge graphs...,https://w3id.org/hi-ontology/scenarios/2/UseCa...,https://w3id.org/hi-thesaurus/CollaborativeMan...,Collaborative Manipulation Use Case,A use case where Hybrid Intelligence systems c...
1,https://w3id.org/hi-ontology/scenarios/4/TaskK...,Knowledge Query Task,A task that involves querying knowledge graphs...,https://w3id.org/hi-ontology/scenarios/4/UseCa...,https://w3id.org/hi-thesaurus/AutonomousTutori...,Autonomous Tutoring Use Case,A use case where Hybrid Intelligence systems p...
2,https://w3id.org/hi-ontology/scenarios/6/TaskL...,Learn Energy Behavior Task,A task that involves learning energy behavior.,https://w3id.org/hi-ontology/scenarios/6/UseCa...,https://w3id.org/hi-thesaurus/EnergyNegotiatio...,Energy Negotiation Use Case,A use case where Hybrid Intelligence systems e...
3,https://w3id.org/hi-ontology/scenarios/1/TaskE...,Explanation Task,A task that involves providing explanations or...,https://w3id.org/hi-ontology/scenarios/1/UseCa...,https://w3id.org/hi-thesaurus/PersonalAssistan...,Personal Assistant Use Case,A use case where Hybrid Intelligence systems f...
4,https://w3id.org/hi-ontology/scenarios/2/TaskE...,Explanation Task,A task that involves providing explanations or...,https://w3id.org/hi-ontology/scenarios/2/UseCa...,https://w3id.org/hi-thesaurus/CollaborativeMan...,Collaborative Manipulation Use Case,A use case where Hybrid Intelligence systems c...
5,https://w3id.org/hi-ontology/scenarios/3/TaskE...,Explanation Task,A task that involves providing explanations or...,https://w3id.org/hi-ontology/scenarios/3/UseCa...,https://w3id.org/hi-thesaurus/GroupDeliberatio...,Group Deliberation Use Case,A use case where Hybrid Intelligence systems f...
6,https://w3id.org/hi-ontology/scenarios/4/TaskE...,Explanation Task,A task that involves providing explanations or...,https://w3id.org/hi-ontology/scenarios/4/UseCa...,https://w3id.org/hi-thesaurus/AutonomousTutori...,Autonomous Tutoring Use Case,A use case where Hybrid Intelligence systems p...
7,https://w3id.org/hi-ontology/scenarios/5/TaskE...,Explanation Task,A task that involves providing explanations or...,https://w3id.org/hi-ontology/scenarios/5/UseCa...,https://w3id.org/hi-thesaurus/MedicalDiagnosis...,Medical Diagnosis Use Case,A use case where Hybrid Intelligence systems a...
8,https://w3id.org/hi-ontology/scenarios/6/TaskE...,Explanation Task,A task that involves providing explanations or...,https://w3id.org/hi-ontology/scenarios/6/UseCa...,https://w3id.org/hi-thesaurus/EnergyNegotiatio...,Energy Negotiation Use Case,A use case where Hybrid Intelligence systems e...
9,https://w3id.org/hi-ontology/scenarios/7/TaskE...,Explanation Task,A task that involves providing explanations or...,https://w3id.org/hi-ontology/scenarios/7/UseCa...,https://w3id.org/hi-thesaurus/VirtualMuseumGui...,Virtual Museum Guide Use Case,A use case where Hybrid Intelligence systems s...


#### 🤖 CQ 3 🤖

**What candidate team compositions satisfy all capability requirements for achieving a specific goal?**

In [59]:
cq3 = """
PREFIX hi:    <https://w3id.org/hi-ontology#>
PREFIX hint:  <https://w3id.org/hi-thesaurus/>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?team ?agent ?agentCapConcept ?agentCapLabel
WHERE {

  VALUES ?goalConcept { hint:FairAccurateVerdictGoal }

  ?goal hi:hasGoalConcept ?goalConcept ;
      hi:requiresTask ?task .
  ?task hi:requiresCapability ?reqCap .
  ?reqCap hi:hasCapabilityConcept ?reqCapConcept .
  ?goalConcept skos:prefLabel ?goalConceptLabel . 
  ?reqCapConcept skos:prefLabel ?reqCapLabel . 
  ?team a hi:HITeam .
  
  FILTER NOT EXISTS {
    ?goal hi:requiresTask ?t2 .
    ?t2 hi:requiresCapability ?cap2 .
    ?cap2 hi:hasCapabilityConcept ?reqCapConcept2 .
    FILTER NOT EXISTS {
      ?team hi:hasMember ?a2 .
      ?a2 hi:hasCapability ?a2Cap .
      ?a2Cap hi:hasCapabilityConcept ?reqCapConcept2 .
    }
  }

  ?team hi:hasMember ?agent .
  ?agent hi:hasCapability ?agentCap .
  ?agentCap hi:hasCapabilityConcept ?agentCapConcept .
  ?agentCapConcept skos:prefLabel ?agentCapLabel . 
}
"""

results_cq3 = g.query(cq3)
df_cq3 = pd.DataFrame([(str(row.team), str(row.agent), str(row.agentCapConcept), str(row.agentCapLabel)) for row in results_cq3],
                        columns=["Team", "Agent", "Agent Capability Concept", "Agent Capability Label"])
df_cq3

,Team,Agent,Agent Capability Concept,Agent Capability Label
0,https://w3id.org/hi-ontology/scenarios/3/HITea...,https://w3id.org/hi-ontology/scenarios/3/AIDel...,https://w3id.org/hi-thesaurus/BayesianReasonin...,Bayesian reasoning capability
1,https://w3id.org/hi-ontology/scenarios/3/HITea...,https://w3id.org/hi-ontology/scenarios/3/AIDel...,https://w3id.org/hi-thesaurus/MetaAnalysisCapa...,Meta-analysis capability
2,https://w3id.org/hi-ontology/scenarios/3/HITea...,https://w3id.org/hi-ontology/scenarios/3/AIDel...,https://w3id.org/hi-thesaurus/ExplainingCapabi...,Explaining capability
3,https://w3id.org/hi-ontology/scenarios/3/HITea...,https://w3id.org/hi-ontology/scenarios/3/Human...,https://w3id.org/hi-thesaurus/SelfReportingCap...,Self-reporting capability
4,https://w3id.org/hi-ontology/scenarios/3/HITea...,https://w3id.org/hi-ontology/scenarios/3/Human...,https://w3id.org/hi-thesaurus/SelfReportingCap...,Self-reporting capability
5,https://w3id.org/hi-ontology/scenarios/4/HITea...,https://w3id.org/hi-ontology/scenarios/4/AITutor,https://w3id.org/hi-thesaurus/BayesianReasonin...,Bayesian reasoning capability
6,https://w3id.org/hi-ontology/scenarios/4/HITea...,https://w3id.org/hi-ontology/scenarios/4/AITutor,https://w3id.org/hi-thesaurus/SemanticReasonin...,Semantic reasoning capability
7,https://w3id.org/hi-ontology/scenarios/4/HITea...,https://w3id.org/hi-ontology/scenarios/4/AITutor,https://w3id.org/hi-thesaurus/KnowledgeGraphQu...,Knowledge graph querying capability
8,https://w3id.org/hi-ontology/scenarios/4/HITea...,https://w3id.org/hi-ontology/scenarios/4/AITutor,https://w3id.org/hi-thesaurus/ExplainingCapabi...,Explaining capability
9,https://w3id.org/hi-ontology/scenarios/4/HITea...,https://w3id.org/hi-ontology/scenarios/4/Human...,https://w3id.org/hi-thesaurus/SelfReportingCap...,Self-reporting capability


#### 🤖 CQ 4 🤖

**How can we infer team eligibility, task assignment, or contextual influences?**

In [ ]:
cq4 = """
PREFIX hi: <https://w3id.org/hi-ontology#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX hi-s1: <https://w3id.org/hi-ontology/scenarios/1>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?constraintLabel ?constraintComment
WHERE {
    VALUES ?goal { hi-s1:GoalUserWellbeing }
    
    ?team           hi:hasGoal              ?goal ;
                    hi:hasEthicalConstraint ?constraint .
    ?goal           rdfs:label              ?goalLabel .
    ?constraint     rdfs:label              ?constraintLabel ;
                    rdfs:comment            ?constraintComment .
}
"""

results_cq4 = g.query(cq4)
df_cq4 = pd.DataFrame([(str(row.constraintLabel), str(row.constraintComment)) for row in results_cq4],
                        columns=["Constraint Label ", "Constraint Comment"])
df_cq4

,Constraint Label,Constraint Comment
0,Privacy Constraint.,User data must be collected and processed resp...
1,Autonomy Constraint.,The assistant must respect user autonomy in li...
2,Transparency Constraint.,The assistant must explain reasoning processes...


#### 🤖 CQ 5 🤖

**What types of interactions occur between agents during the execution of a specific task?**

In [ ]:
cq5 = """
PREFIX hi: <https://w3id.org/hi-ontology#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX hi-s1: <https://w3id.org/hi-ontology/scenarios/1>

SELECT DISTINCT ?methodLabel ?methodComment ?goalLabel ?goalComment
WHERE {
    VALUES ?task { hi-s1:TaskConflictResolution }
    
    ?taskExecution  hi:realizesTask         ?task ;
                    hi:usesMethod           ?method ;
                    hi:towardsGoal          ?goal .
    ?method         rdfs:label              ?methodLabel ;
                    rdfs:comment            ?methodComment .
    ?goal           rdfs:label              ?goalLabel ;
                    rdfs:comment            ?goalComment .             

}
"""

results_cq5 = g.query(cq5)
df_cq5 = pd.DataFrame([(str(row.methodLabel), str(row.methodComment), str(row.goalLabel), str(row.goalComment)) for row in results_cq5],
                        columns=["Method Label ", "Method Comment", "Goal Label ", "Goal Comment"])
df_cq5

,Method Label,Method Comment,Goal Label,Goal Comment
0,Argumentation-based Reasoning Method.,Argumentation-based reasoning used by the assi...,User Wellbeing Goal.,Maintain and improve user health habits throug...


#### 🤖 CQ 6 🤖

**Given a method, which goals can be achieved with it, and under which task configurations does it apply?**

In [ ]:
cq6 = """
PREFIX hi: <https://w3id.org/hi-ontology#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX hi-s1: <https://w3id.org/hi-ontology/scenarios/1>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?evaluationLabel ?evaluationComment ?methodLabel ?methodComment ?goalLabel ?goalComment
WHERE {
    VALUES ?team { hi-s1:HITeamPersonalAssistant }

    ?team           hi:operatesInContext    ?context .
    ?context        hi:influences           ?evaluation .
    ?evaluation     a                       hi:Evaluation ;   
                    rdfs:label              ?evaluationLabel ;
                    rdfs:comment            ?evaluationComment ;
                    hi:usesMetric           ?metric ;
                    hi:hasExperiment        ?experiment .
    ?metric         rdfs:label              ?methodLabel ;
                    rdfs:comment            ?methodComment .
    ?experiment     rdfs:label              ?goalLabel ;
                    rdfs:comment            ?goalComment .  

}
"""

results_cq6 = g.query(cq6)
df_cq6 = pd.DataFrame([(str(row.evaluationLabel), str(row.evaluationComment), 
                        str(row.methodLabel), str(row.methodComment), 
                        str(row.goalLabel), str(row.goalComment)) for row in results_cq6],
                        columns=["Evaluation Label", "Evaluation Comment", "Method Label ", "Method Comment", "Goal Label ", "Goal Comment"])
df_cq6

,Evaluation Label,Evaluation Comment,Method Label,Method Comment,Goal Label,Goal Comment
0,Evaluation of Transparency Impact on User Trust.,Evaluates how providing explanations affects u...,Trust Preservation Metric.,Quantifies user trust retention when explanati...,Experiment on Transparency Impact on User Trust.,Experiment to assess how explanations affect u...


#### 🤖 CQ 7 🤖

**Given an evaluation metric, which task can be assessed with it?**

#### 🤖 CQ 8 🤖

**Given a goal, which experiments test which metrics, and what hypotheses do these experiments address?**

#### 🤖 CQ 9 🤖

**Given a contextual constraint, which use cases are associated with it, and how does it affect the team?**

#### 🤖 CQ 10 🤖

**Which constraints are shared across different use cases and agents?**

#### 🤖 CQ 11 🤖

**Given a use case, which contextual phenomena, such as bias or feedback effects, are associated with it?**

#### 🤖 CQ 12 🤖

**Under what conditions does an agent's behaviour comply with or violate a contextual constraint associated with a use case?**